In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


##Data Preparation##

In [0]:
# EGD cell
import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 

# img_width = 475
# img_height = 388
img_width = 224
img_height = 224
batch_size = 1
channels = 3

#EGD data
egd_dir = "/gdrive/My Drive/EGD/"

egd_datagen = ImageDataGenerator(rescale=1./255)

egd_generator = egd_datagen.flow_from_directory(
    egd_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)

features = np.zeros(shape=(102, img_height, img_width, 3))
labels = np.zeros(shape=(102))
i = 0

for inputs_batch, labels_batch in egd_generator:
    features[i * batch_size : (i + 1) * batch_size] = inputs_batch
    labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= 102:
        break   
        
print(features.shape, labels.shape)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

# if Alexnet
# X_train = np.rollaxis(X_train, 3, 1)  
# X_test = np.rollaxis(X_test, 3, 1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

##VGG16 pretrained##

In [19]:
import numpy as np 
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Softmax, Flatten, Dense, BatchNormalization 
from keras.metrics import categorical_accuracy
from keras import backend as K
from keras import regularizers
import tensorflow as tf
from keras.models import Sequential
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
from keras import layers
from keras.callbacks import TensorBoard, Callback
from keras.applications import VGG16, resnet50
from keras import optimizers
import keras


class MetricsCheckpoint(Callback):
    """Callback that saves metrics after each epoch"""
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)


base_model = VGG16(weights='imagenet',
                  include_top=True,
                  input_shape=(img_height, img_width, channels))

base_model.summary()

model = Model(inputs=base_model.input, outputs=base_model.get_layer('predictions').output)

X_train_1k_features = model.predict(X_train)
X_test_1k_features =  model.predict(X_test)
print(X_train_1k_features.shape)
print(X_test_1k_features.shape)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score

clf = RandomForestClassifier(n_estimators=100, max_depth=12, random_state=0, bootstrap=True)
clf.fit(X_train_1k_features, y_train)

y_pred = clf.predict(X_test_1k_features) 
aroc = roc_auc_score(y_test, y_pred)

test_score = clf.score(X_test_1k_features, y_test)

print("test precision:", test_score)
print("test AROC:", aroc)

test precision: 0.8571428571428571
test AROC: 0.8545454545454546


##AlexNet pretrained##

In [72]:
# !git clone https://github.com/heuritech/convnets-keras.git
# !cd convnets-keras
# !python setup.py install
! ls convnets-keras/
! cd convnets-keras;python setup.py install


convnetskeras  examples  LICENSE.txt  README.md  setup.py
running install
running build
running build_py
creating build
creating build/lib
creating build/lib/convnetskeras
copying convnetskeras/__init__.py -> build/lib/convnetskeras
copying convnetskeras/imagenet_tool.py -> build/lib/convnetskeras
copying convnetskeras/customlayers.py -> build/lib/convnetskeras
copying convnetskeras/convnets.py -> build/lib/convnetskeras
creating build/lib/convnetskeras/data
copying convnetskeras/data/meta_clsloc.mat -> build/lib/convnetskeras/data
running install_lib
creating /usr/local/lib/python3.6/dist-packages/convnetskeras
copying build/lib/convnetskeras/__init__.py -> /usr/local/lib/python3.6/dist-packages/convnetskeras
creating /usr/local/lib/python3.6/dist-packages/convnetskeras/data
copying build/lib/convnetskeras/data/meta_clsloc.mat -> /usr/local/lib/python3.6/dist-packages/convnetskeras/data
copying build/lib/convnetskeras/imagenet_tool.py -> /usr/local/lib/python3.6/dist-packages/convnets

In [73]:
from convnetskeras import convnets
model = convnet('alexnet',weights_path="alexnet_weights.h5", heatmap=False)


ImportError: ignored

In [74]:
import numpy as np
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Input
# from keras.layers import concaten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.models import Model
from keras.models import Sequential
from keras.optimizers import SGD
from keras import backend as K
from keras.engine import Layer
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Lambda
from keras.layers import concatenate

def splittensor(axis=1, ratio_split=1, id_split=0, **kwargs):
    def f(X):
        div = X.shape[axis] // ratio_split

        if axis == 0:
            output = X[id_split * div:(id_split + 1) * div, :, :, :]
        elif axis == 1:
            output = X[:, id_split * div:(id_split + 1) * div, :, :]
        elif axis == 2:
            output = X[:, :, id_split * div:(id_split + 1) * div, :]
        elif axis == 3:
            output = X[:, :, :, id_split * div:(id_split + 1) * div]
        else:
            raise ValueError('This axis is not possible')

        return output

    def g(input_shape):
        output_shape = list(input_shape)
        output_shape[axis] = output_shape[axis] // ratio_split
        return tuple(output_shape)

    return Lambda(f, output_shape=lambda input_shape: g(input_shape), **kwargs)

def crosschannelnormalization(alpha=1e-4, k=2, beta=0.75, n=5, **kwargs):
    """
    This is the function used for cross channel normalization in the original
    Alexnet
    """

    def f(X):
        b, ch, r, c = X.shape
        half = n // 2
        square = K.square(X)
        extra_channels = K.spatial_2d_padding(K.permute_dimensions(square, (0, 2, 3, 1))
                                              , (0, half))
        extra_channels = K.permute_dimensions(extra_channels, (0, 3, 1, 2))
        scale = k
        for i in range(n):
            scale += alpha * extra_channels[:, i:i + ch, :, :]
        scale = scale ** beta
        return X / scale

    return Lambda(f, output_shape=lambda input_shape: input_shape, **kwargs)


def AlexNet(weights_path=None):
    
    inputs = Input(shape=(227, 227, 3))
    conv_1 = Convolution2D(96, 11, 11, subsample=(4, 4), activation='relu',name='conv_1')(inputs)

    conv_2 = MaxPooling2D((3, 3), strides=(2, 2))(conv_1)
    conv_2 = crosschannelnormalization(name='convpool_1')(conv_2)
    conv_2 = ZeroPadding2D((2, 2))(conv_2)
    conv_2 = concatenate([Convolution2D(128, 5, 5, activation='relu', name='conv_2_' + str(i + 1))(splittensor(ratio_split=2, id_split=i)(conv_2)) for i in range(2)], mode='concat', concat_axis=1, name='conv_2')

    conv_3 = MaxPooling2D((3, 3), strides=(2, 2))(conv_2)
    conv_3 = crosschannelnormalization()(conv_3)
    conv_3 = ZeroPadding2D((1, 1))(conv_3)
    conv_3 = Convolution2D(384, 3, 3, activation='relu', name='conv_3')(conv_3)

    conv_4 = ZeroPadding2D((1, 1))(conv_3)
    conv_4 = concatenate([Convolution2D(192, 3, 3, activation='relu', name='conv_4_' + str(i + 1))(splittensor(ratio_split=2, id_split=i)(conv_4)) for i in range(2)], mode='concat', concat_axis=1, name='conv_4')

    conv_5 = ZeroPadding2D((1, 1))(conv_4)
    conv_5 = concatenate([Convolution2D(128, 3, 3, activation='relu', name='conv_5_' + str(i + 1))(splittensor(ratio_split=2, id_split=i)(conv_5)) for i in range(2)], mode='concat', concat_axis=1, name='conv_5')

    dense_1 = MaxPooling2D((3, 3), strides=(2, 2), name='convpool_5')(conv_5)

    dense_1 = Flatten(name='flatten')(dense_1)
    dense_1 = Dense(4096, activation='relu', name='dense_1')(dense_1)
    dense_2 = Dropout(0.5)(dense_1)
    dense_2 = Dense(4096, activation='relu', name='dense_2')(dense_2)
    dense_3 = Dropout(0.5)(dense_2)
    dense_3 = Dense(1000, name='dense_3')(dense_3)
    prediction = Activation('softmax', name='softmax')(dense_3)
    model = Model(input=inputs, output=prediction)

    if weights_path:
        model.load_weights(weights_path)

    return model

alex_weight_path = 'alexnet_weights.h5'

base_model = AlexNet(alex_weight_path)
base_model.summary()

# model = Model(inputs=base_model.input, outputs=base_model.get_layer('predictions').output)
# X_train_1k_features = model.predict(X_train)
# X_test_1k_features =  model.predict(X_test)
# print(X_train_1k_features.shape)
# print(X_test_1k_features.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:69: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (11, 11), activation="relu", name="conv_1", strides=(4, 4))`


TypeError: ignored

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score

clf = RandomForestClassifier(n_estimators=100, max_depth=12, random_state=0, bootstrap=True)
clf.fit(X_train_1k_features, y_train)

y_pred = clf.predict(X_test_1k_features) 
aroc = roc_auc_score(y_test, y_pred)

test_score = clf.score(X_test_1k_features, y_test)

print("test precision:", test_score)
print("test AROC:", aroc)